In [ ]:
# https://jalammar.github.io/illustrated-gpt2/
# GPT 작동 원리
# GPT2는 한번에 하나의 token을 출력함

In [ ]:
# https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-2.%20kogpt2_text_generation_gpu.ipynb

# KoGPT2로 문장 생성

In [7]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [8]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

In [9]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [10]:
sent = '근육이 커지기 위해서는'

In [11]:
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

tf.Tensor([[33245 10114 12748 11357]], shape=(1, 4), dtype=int32)


In [12]:
output = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        use_cache=True)
output_ids = output.numpy().tolist()[0]
print(output_ids)

[33245, 10114, 12748, 11357, 23879, 39306, 9684, 7884, 10211, 15177, 26421, 387, 17339, 7889, 9908, 15768, 6903, 15386, 8146, 12923, 9228, 18651, 42600, 9564, 17764, 9033, 9199, 14441, 7335, 8704, 12557, 32030, 9510, 18595, 9025, 10571, 25741, 10599, 13229, 9508, 7965, 8425, 33102, 9122, 21240, 9801, 32106, 13579, 12442, 13235, 19430, 8022, 12972, 9566, 11178, 9554, 24873, 7198, 9391, 12486, 8711, 9346, 7071, 36736, 9693, 12006, 9038, 10279, 36122, 9960, 8405, 10826, 18988, 25998, 9292, 7671, 9465, 7489, 9277, 10137, 9677, 9248, 9912, 12834, 11488, 13417, 7407, 8428, 8137, 9430, 14222, 11356, 10061, 9885, 19265, 9377, 20305, 7991, 9178, 9648, 9133, 10021, 10138, 30315, 21833, 9362, 9301, 9685, 11584, 9447, 42129, 10124, 7532, 17932, 47123, 37544, 9355, 15632, 9124, 10536, 13530, 12204, 9184, 36152, 9673, 9788, 9029, 11764]


In [13]:
tokenizer.decode(output_ids)

'근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.\n특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.\n또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.\n아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.\n운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.\n운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.\n운동을'

# TOP 5 추출

In [14]:
import numpy as np

In [15]:
output = model(input_ids)

In [16]:
output.logits

<tf.Tensor: shape=(1, 4, 51200), dtype=float32, numpy=
array([[[-5.2999744 , -5.247863  , -4.796324  , ...,  0.02473605,
         -1.145249  , -0.6509337 ],
        [-4.790443  , -5.4123983 , -4.876633  , ..., -0.9312227 ,
         -3.6518145 , -5.819604  ],
        [-4.0496645 , -3.5694022 , -5.737194  , ..., -2.4115632 ,
         -2.4622674 , -3.5288236 ],
        [-5.456258  , -6.02514   , -6.1407743 , ..., -0.22475755,
         -4.0253325 , -3.2478456 ]]], dtype=float32)>

In [17]:
output.logits.shape

TensorShape([1, 4, 51200])

In [18]:
top5 = tf.math.top_k(output.logits[0, -1], k=5)

In [19]:
tokenizer.convert_ids_to_tokens(top5.indices.numpy())

['▁무엇보다', '▁우선', '▁반드시', '▁피부', '▁무엇보다도']

# 문장 생성

In [20]:
sent = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(sent)

In [21]:
import random

while len(input_ids) < 50:
    output = model(np.array([input_ids]))
    top5 = tf.math.top_k(output.logits[0, -1], k=5)
    token_id = random.choice(top5.indices.numpy())
    input_ids.append(token_id)

In [22]:
tokenizer.decode(input_ids)

'근육이 커지기 위해서는 무엇보다도 균형감각을 유지하는데 힘써주어야 할 것이다.\n특히 요즘같이 날씨로 쌀뜨물 같은 날씨에도 쉽게 잠이 드는 계절이 되면 잠들기는 쉽지 않을 것이다.</d> 이 같은 현상은 올해도 지속돼 올해는 전 세계'

In [23]:
# https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-3.%20kogpt2_chatbot_gpu.ipynb


In [24]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [25]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='', eos_token='', pad_token='')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.4.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.2.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [26]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.pad_token_id)
print('-' * 10)
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

51200
51200
51200
----------
</s>
<usr>
<pad>
<sys>


In [27]:
import pandas as pd
import tqdm
import urllib.request

In [28]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [29]:
len(train_data)

11823

In [30]:
batch_size = 32

In [31]:
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('' + question + '' + answer) 
    yield bos_token + sent + eos_token

In [32]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [33]:
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [34]:
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[51200  9349  7888   739  7318   376 25000  6824  9108  9028  7098 25856
  51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9020  8263  7497 10192 11615  8210  8006 11567  8711  9535  7483
  12521 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9085  7597   395  8149 10624  7397 24224 13358  7182  8030 19138
  16899  9677  8234   389 51200 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9085  7597   395  8149  9465 10624  7397 24224 13358  7182  8030
  19138 16899  9677  8234   389 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9943   422   418  9327  8702  7098  7141 16912 18328  8671  7415
   8263  8234   389 51200 51200 51200 51200 51200 51200 51200 51200 51200
  51200 51200 51200 51200]
 [51200  9815   410 21249 10174  6824  8210  8006 16146 11056 11594 10137
  10556  9266  8711 25856 51200 51200 51

In [35]:
tokenizer.decode(batch[0])

'<|endoftext|> 12시 땡!하루가 또 가네요.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'

In [36]:
print(batch[0])

tf.Tensor(
[51200  9349  7888   739  7318   376 25000  6824  9108  9028  7098 25856
 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200 51200
 51200 51200 51200 51200], shape=(28,), dtype=int32)


In [37]:
print(tokenizer.encode(' 12시 땡! 하루가 또 가네요.'))

[9349, 7888, 739, 7318, 376, 12557, 6824, 9108, 9028, 7098, 25856]


In [38]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [39]:
steps = len(train_data) // batch_size + 1
print(steps)

370


In [ ]:
EPOCHS = 3

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

In [42]:
text = '오늘도 좋은 하루!'

In [43]:
sent = '' + text + ''

In [44]:
input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])

In [ ]:
output = model.generate(input_ids, max_length=50, early_stopping=True, eos_token_id=tokenizer.eos_token_id)

In [ ]:
decoded_sentence = tokenizer.decode(output[0].numpy().tolist())

In [ ]:
decoded_sentence.split(' ')[1].replace('', '')

In [ ]:
output = model.generate(input_ids, max_length=50, do_sample=True, top_k=10)
tokenizer.decode(output[0].numpy().tolist())

In [49]:
def return_answer_by_chatbot(user_text):
  sent = '' + user_text + ''
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split(' ')[1].replace('', '')
  return chatbot_response

In [ ]:
return_answer_by_chatbot('안녕! 반가워~')

In [ ]:
# 형식에 맞는 텍스트 데이터 준비, 모델 생성, 모델 활용
# https://platform.openai.com/docs/guides/fine-tuning 파인튜닝 가이드